In [ ]:
import eikon as ek
import pandas as pd
import numpy as np
import time
import datetime
import matplotlib.pyplot as plt
import os
import sqlite3
from sqlite3 import Error
from scipy import stats
import scipy as sp
import statsmodels.api as sm
import pylab
import statistics
from statsmodels.tsa.stattools import adfuller
from openpyxl import load_workbook
from openpyxl import Workbook
from datetime import date
from OLS import OLS

ek.set_app_key('ed89d084197d408686e3cfa56e3356daad46c055')

print('Done')

ticker = ['BTC=','LTC=','ETH=', 'BCH=', 'XRP=','EUR=','CHF=','GBP=','JPY=','CNY=','CAD=', 'AUD=','MXN=','CHFEUR=R', 'GBPEUR=R', 'JPYEUR=R','CNYEUR=R','CADEUR=R','AUDEUR=R','MXNEUR=R'] #wäre gut, wenn man sie nicht manuell eingeben muss, sondern aus einer Liste ziehen.
print('Retrieving Data for',ticker)

df1 = ek.get_timeseries(ticker, fields = 'CLOSE',interval = 'daily', calendar = 'tradingdays', start_date = '2019-01-01', end_date = '2019-04-11')
df2 = ek.get_timeseries(ticker, fields = 'CLOSE',interval = 'daily', calendar = 'tradingdays', start_date = '2019-04-12', end_date = '2019-09-08')
df3 = ek.get_timeseries(ticker, fields = 'CLOSE',interval = 'daily', calendar = 'tradingdays', start_date = '2019-09-09', end_date = '2020-02-05')
df4 = ek.get_timeseries(ticker, fields = 'CLOSE',interval = 'daily', calendar = 'tradingdays', start_date = '2020-02-06', end_date = '2020-07-04')
df5 = ek.get_timeseries(ticker, fields = 'CLOSE',interval = 'daily', calendar = 'tradingdays', start_date = '2020-07-05', end_date = '2020-12-01')
df6 = ek.get_timeseries(ticker, fields = 'CLOSE',interval = 'daily', calendar = 'tradingdays', start_date = '2020-12-02' , end_date = '2021-04-30')

df = pd.concat([df1,df2,df3,df4,df5,df6], axis = 0)

df.columns = df.columns.str.replace('=', '')

#Save data to an excel sheet
df.to_excel('AQM_Data.xlsx')
print('Retrieving Data successful.')


df_p = pd.read_excel('AQM_Data.xlsx')
df_p = df_p.set_index(df_p['Date'])
df_p = df_p.drop(axis=1, columns = 'Date')

#Cleaning Python Data

df_clean=pd.DataFrame(df_p)
df_clean.dropna(axis='rows',inplace=True)

print('table size before cleaning:', df.shape)
print('table size after cleaning:', df_clean.shape)

df_p = df_clean
df = df_p

#Read & clean excel Data

df_e = pd.read_excel('AQM_task_3_v3.xlsx')
df_e = df_e.set_index(df_e['Date'])
df_e = df_e.drop(axis=1, columns = 'Date')

df_clean2=pd.DataFrame(df_e)
df_clean2.dropna(axis='rows',inplace=True)

print('table size before cleaning:', df.shape)
print('table size after cleaning:', df_clean.shape)

df_e = df_clean2

#Check if the values are the same

ticker_new = [s.replace("=", "") for s in ticker]

for i in ticker_new:
    s = df_e [i].sum() - df_p[i].sum()
    m = df_e [i].mean() - df_p[i].mean()
    min = df_e [i].min() - df_p[i].min()
    max = df_e [i].max() - df_p[i].max()
    std = df_e [i].std() - df_p[i].std()
    if (s + m + min + max + std) == 0:
        print(i,': Data are equal.')        
    else:
        print (i,': Data are NOT equal.')
        
        
#Create a folder for the Charts, Histograms etc.

def create_folder(directory):
    try:
        if os.path.exists(directory):
            print("Folder '%s' already exists." %directory, "\nFiles will be stored in '%s'." %directory)
        if not os.path.exists(directory):
            os.makedirs(directory)
            print("Folder '%s' has been created." %directory, "\nFiles will be stored in '%s'." %directory)
    except OSError:
        print("Creation of the folder failed.")
        
create_folder('Charts')

def plots_save():
    for col in df.iteritems():
        print('Plotting '+col[0])
        fig, ax = plt.subplots()
        indicator= df[col[0]]
        ax.plot(indicator, alpha=0.9, color='blue')
        plt.title(col[0])
        plt.savefig('Charts\Plot_'+col[0]+'.png')
        plt.close()
        
plots_save()

#Check if values are missing 


ticker_new = [s.replace("=", "") for s in ticker]

for i in ticker_new:
    print('')
    if (df_e[i].isnull().sum().any() == False) and (df_p[i].isnull().sum().any() == False):
        print(i,': There isn\'t any wrong or missing data.')
        print('No cleaning needed.')
    else:
        if df_e[i].isnull().sum().any() == True:          
            print(i,': There is',df_e[i].isnull().sum(),'wrong or missing data in the Excel data!')

            print('Table size before cleaning:', df_e.shape)
            df_e.dropna(axis = 'rows', how = 'any', inplace = True)

            print('Table size after cleaning:', df_e.shape)
        else:          
            print(i,': There is',df_p[i].isnull().sum(),'wrong or missing data in the Python data!')

            print('Table size before cleaning:', df_p.shape)
            df_p.dropna(axis = 'rows', how = 'any', inplace = True)

            print('Table size after cleaning:', df_p.shape)

print('\nSince the data is identical, we can work only with the data from python (df_p = df)')


#Initial Analysis

df.describe()

